In [1]:
import json
import logging
import os
import pandas as pd
import random
import sys
from datetime import date
from dotenv import load_dotenv
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from utils.utils import GPT

In [2]:
# params
# st_mdl = "gpt-3.5-turbo-1106" # model string
st_mdl = "gpt-4-1106-preview"
st_task = "mmlu" # name of task

# logging
## today's date
st_today = date.today().strftime("%Y_%m_%d")

## file name:
st_fn = f"{st_today}_{st_task}_n200_{st_mdl}"

## Configure the logger
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

## Create a file handler and set the formatter
file_handler = logging.FileHandler(f"results/{st_fn}.log")
file_handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
file_handler.setFormatter(formatter)

## Create a logger instance and add the file handler
logger = logging.getLogger(__name__)
logger.addHandler(file_handler)

In [3]:
# load api key from .env and create GPT object
load_dotenv()  # Load environment variables from .env file
openai_api_key = os.getenv("OPENAI_API_KEY")
gpt = GPT(openai_api_key, logger=logger, model=st_mdl)

# Load into a dictionary
data = {}
subjects = [fn[:-9] for fn in os.listdir('data/test') if fn.endswith('_test.csv')]
for subj in subjects:
    data[subj] = pd.read_csv(f"data/test/{subj}_test.csv", header=None).to_numpy()
    # data.append(json.loads(f)) # keys in each row: ['_id', 'answer', 'question', 'supporting_facts', 'context', 'type', 'level']

# data = pd.read_csv('data/24game.csv').to_dict(orient='records') # load CSV into list of dicts

In [4]:
def compare_strings(str1, str2):
    """
    This function compares two strings after converting them to lowercase and stripping leading and trailing spaces.
    """
    return str1.lower().strip() == str(str2).lower().strip()


def check_mcq(pred, gold):
    """
        Check if prediction for multiple choice question is correct 
    """
    return pred.lower().strip()[0] == gold.lower()


def check_bool(str1, bool2):
    """
        Checks if string str1 equal to bool bool2
    """
    try:
        if bool(str1) == bool2:
            return True
    except:
        pass

    return False
        

In [5]:
# dataset-specific adjustments (we eventually want a list of dicts of format [{"id": "xx", "question": "xxx", "context": "xxx"}, ...]

# pre-process data
# for d in data:
#     d['question'] = d['Puzzles']
print(len(subjects))
print(data['anatomy'][0])
df = pd.DataFrame(data = data['anatomy'])
df.head()

57
['A lesion causing compression of the facial nerve at the stylomastoid foramen will cause ipsilateral'
 'paralysis of the facial muscles.'
 'paralysis of the facial muscles and loss of taste.'
 'paralysis of the facial muscles, loss of taste and lacrimation.'
 'paralysis of the facial muscles, loss of taste, lacrimation and decreased salivation.'
 'A']


,0,1,2,3,4,5
0,A lesion causing compression of the facial ner...,paralysis of the facial muscles.,paralysis of the facial muscles and loss of ta...,"paralysis of the facial muscles, loss of taste...","paralysis of the facial muscles, loss of taste...",A
1,"A ""dished face"" profile is often associated with",a protruding mandible due to reactivation of t...,a recessive maxilla due to failure of elongati...,an enlarged frontal bone due to hydrocephaly.,defective development of the maxillary air sinus.,B
2,Which of the following best describes the stru...,Bladder,Kidney,Ureter,Urethra,A
3,Which of the following structures is derived f...,Motor neurons,Skeletal muscles,Melanocytes,Sweat glands,C
4,Which of the following describes the cluster o...,Afferent arteriole,Glomerulus,Loop of Henle,Renal pelvis,B


## Generator

In [6]:
predictions = [] # list of predictions, reset after each set of runs
indices = [] # list of question indices
answers = [] # list of gold answers
evals = [] # list of whether prediction matches answer
questions = [] # list of questions
lst_subjects = [] # list of subjects for each row
lst_indices = [] # list of indices for each row. These indices are subject-specific

# Evaluate
num_questions = 10 # number of questions to run per category
for subj in subjects:
    subj_data = data[subj]
    indices = random.sample(range(len(subj_data)), num_questions)
    for idx in indices:
        item = subj_data[idx]
        lst_subjects.append(subj)
        lst_indices.append(idx)
        
        question, choices, gold = item[0], item[1:5], item[5]
        question = f"{question}\nA. {choices[0]}\nB. {choices[1]}\nC. {choices[2]}\nD. {choices[3]}"
        logger.info(question)
        
        msg = [{"role": "system", "content": f"The following are multiple choice questions (with answers) about {subj}. Respond with only a single letter; one of A, B, C, or D. Do not respond with any other characters or words"}, {"role": "user", "content": question}]
        pred = gpt.gen(msgs=msg).content
        predictions.append(pred)
        answers.append(gold)
        evals.append(check_mcq(predictions[-1], gold))
        questions.append(question)
        print(predictions[-1])

# Print the predictions and answers
logger.info(predictions)
# logger.info(answers)
percent_correct = (sum(evals) / len(evals)) * 100

print(f"Percentage of True values: {percent_correct}%")

# create a dataframe with the data 
df = pd.DataFrame({'Index': lst_indices, 'Subject': lst_subjects, 'Question': questions, 'Prediction': predictions, 'Answer': answers, 'Eval': evals})

# save the datafrme to csv
df.to_csv(f'results/{st_fn}.csv', index=False)

2024-02-14 00:20:14,072 - INFO - Compute the product in the given ring. (2,3)(3,5) in Z_5 x Z_9
A. (1,1)
B. (3,1)
C. (1,6)
D. (3,6)
2024-02-14 00:20:14,499 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-14 00:20:14,501 - INFO - Find all cosets of the subgroup 4Z of 2Z.
A. 4Z
B. 4Z, 2 + 4Z
C. 2Z
D. Z


C


2024-02-14 00:20:14,862 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-14 00:20:14,864 - INFO - Statement 1 | If H is a subgroup of G and a belongs to G then aH is a subgroup of G if and only if a is in H. Statement 2 | If H is a subgroup of G and a and b belong to G then aH = bH if and only if ab is in H.
A. True, True
B. False, False
C. True, False
D. False, True


A.


2024-02-14 00:20:16,353 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-14 00:20:16,355 - INFO - Find the characteristic of the ring Z x Z.
A. 0
B. 3
C. 12
D. 30


...

A


2024-02-14 00:25:44,897 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-14 00:25:44,898 - INFO - Who is the founder of Sikhism?
A. Guru Gobind Singh
B. Guru Nanak
C. Guru Kabir
D. Guru Hargobind


A


2024-02-14 00:25:45,443 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-14 00:25:45,445 - INFO - ['C', 'A.', 'D', 'A', 'C', 'C', 'A. True, True', 'D', 'D', 'C', 'D', 'B.', 'B', 'D', 'D', 'D', 'A', 'C', 'A', 'D', 'B', 'B', 'C', 'D', 'D', 'C', 'B', 'D', 'C', 'B', 'B.', 'C', 'C', 'B', 'D', 'D', 'B. Absolutism, Relativism', 'B.', 'C', 'B', 'D', 'B.', 'C.', 'B.', 'D', 'A', 'D', 'A', 'B.', 'B.', 'D', 'C', 'A', 'D', 'A', 'C', 'C', 'D', 'B.', 'C', 'A', 'B.', 'A.', 'A', 'C.', 'B.', 'A', 'B.', 'B', 'B.', 'B', 'D', 'B', 'D.', 'B.', 'C.', 'B.', 'B.', 'A', 'C', 'C', 'B', 'B.', 'C', 'B.', 'C', 'D', 'B.', 'D', 'C', 'C', 'B', 'D', 'B.', 'C.', 'A', 'B.', 'D', 'D', 'D', 'B', 'B', 'A', 'D', 'B.', 'A', 'B.', 'B.', 'B.', 'D', 'A', 'B. False, False', 'A', 'C', 'D', 'C.', 'A.', 'D', 'B', 'D', 'A.', 'B. 0.8 N', 'B', 'C', 'B.', 'D', 'A.', 'C', 'A.', 'B.', 'A', 'B', 'D', 'A.', 'B.', 'B', 'A', 'B', 'C', 'B', 'B', 'B', 'D', 'A', 'D', 'B.', 'A', 'A', 'A', 'C.', 'B',

B
Percentage of True values: 64.21052631578948%


In [8]:
len(evals)

570

In [7]:
df = pd.read_csv(f"results/{st_fn}.csv")
df['Answer'] = df['Answer'].apply(lambda x: x.split('\n#### ')[-1])
df['Eval'] = df.apply(lambda row: check_mcq(row['Prediction'], row['Answer']), axis=1)

df.to_csv(f'results/{st_fn}.csv', index=False)
df.head()

,Index,Subject,Question,Prediction,Answer,Eval
0,17,abstract_algebra,"Compute the product in the given ring. (2,3)(3...",C,C,True
1,91,abstract_algebra,Find all cosets of the subgroup 4Z of 2Z.\nA. ...,A.,B,False
2,95,abstract_algebra,Statement 1 | If H is a subgroup of G and a be...,D,C,False
3,77,abstract_algebra,Find the characteristic of the ring Z x Z.\nA....,A,A,True
4,25,abstract_algebra,Let G denoted the set of all n x n non-singula...,C,C,True


## Validator

### Baseline

In [ ]:
fn_sfx = "" # filename suffix, used when saving validator file
sys_prompt = "Answer 'TRUE' if the answer is correct and 'FALSE' if the answer is incorrect."

### Search for optimal prompt

In [6]:
fn_sfx = "_prmpt" # filename suffix, used when saving validator file
sys_prompt = "Think step by step. First generate your own answer to the question and then compare this with the provided answer. Check, then double check your thinking. The last word of your response should be 'TRUE' if the answer is correct and 'FALSE' if the answer is incorrect, given the question."

### Validator code

In [8]:
gen_data_fn = f"results/{st_fn}.csv" # change this to the filename of the generator results
# gen_data_fn = 'results/2024_01_27_mmlu_n200_gpt-3.5-turbo-1106.csv'

df = pd.read_csv(gen_data_fn)
indices = df.iloc[:, 0].tolist()
indices = indices[:-1] # just because we manually added an additional row on the CSV when calculating scores

val_preds = [] # list of predictions for validator, reset after each set of runs
val_evals = [] # whether prediction matches answer


# Evaluate
for _, row in df.iterrows():
    ans = row['Prediction']
    # gold = row['Manual Eval']
    gold = row['Eval']
    question = row['Question']
    
    q = "Question: " + question + " "
    q += f"\nAnswer: {ans}"
    logger.info(q)
    msg = [{"role": "system", "content": "The following are a series of multiple choice problems. Respond 'TRUE' if the answer to the question is correct and 'FALSE' if incorrect. Respond only with one word."}, {"role": "user", "content": q}]
    val_preds.append(gpt.gen(msgs=msg).content)
    val_evals.append(check_bool(val_preds[-1], gold))
    print(val_preds[-1])

# Print the predictions and answers
logger.info(val_preds)
logger.info(df['Eval'].tolist())
percent_correct = (sum(val_evals) / len(val_evals)) * 100

print(f"Percentage of True values: {percent_correct}%")

# add validator predictions to column
df['Validator Pred'] = val_preds

# save the dataframe to csv
df.to_csv(f'results/{st_fn}_with_val{fn_sfx}.csv', index=False);

2024-02-14 16:02:54,838 - INFO - Question: Compute the product in the given ring. (2,3)(3,5) in Z_5 x Z_9
A. (1,1)
B. (3,1)
C. (1,6)
D. (3,6) 
Answer: C
2024-02-14 16:02:55,282 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-14 16:02:55,284 - INFO - Question: Find all cosets of the subgroup 4Z of 2Z.
A. 4Z
B. 4Z, 2 + 4Z
C. 2Z
D. Z 
Answer: A.


FALSE


2024-02-14 16:02:55,734 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-14 16:02:55,735 - INFO - Question: Statement 1 | If H is a subgroup of G and a belongs to G then aH is a subgroup of G if and only if a is in H. Statement 2 | If H is a subgroup of G and a and b belong to G then aH = bH if and only if ab is in H.
A. True, True
B. False, False
C. True, False
D. False, True 
Answer: D


FALSE


2024-02-14 16:02:56,359 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-14 16:02:56,361 - INFO - Question: Find the characteristic of the ring Z x Z.
A. 0
B. 3
C. 12
D. 30 
Answer: A


...

FALSE


2024-02-14 16:08:42,475 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-14 16:08:42,477 - INFO - Question: Who is the founder of Sikhism?
A. Guru Gobind Singh
B. Guru Nanak
C. Guru Kabir
D. Guru Hargobind 
Answer: B


TRUE


2024-02-14 16:08:42,792 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-02-14 16:08:42,794 - INFO - ['FALSE', 'FALSE', 'FALSE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'FALSE', 'TRUE', 'TRUE', 'TRUE', 'FALSE', 'FALSE', 'TRUE', 'TRUE', 'TRUE', 'FALSE', 'TRUE', 'TRUE', 'TRUE', 'FALSE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'FALSE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'FALSE', 'TRUE', 'FALSE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'FALSE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'FALSE', 'FALSE', 'TRUE', 'TRUE', 'FALSE', 'TRUE', 'TRUE', 'FALSE', 'TRUE', 'TRUE', 'TRUE', 'FALSE', 'TRUE', 'FALSE', 'FALSE', 'FALSE', 'FALSE', 'TRUE', 'FALSE', 'TRUE', 'FALSE', 'TRUE', 'TRUE', 'TRUE', 'FALSE', 'TRUE', 'TRUE', 'FALSE', 'TRUE', 'FALSE', 'TRUE', 'TRUE', 'TRUE', 'FALSE', 'TRUE', 'TRUE', 'FALSE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', 'TRUE', '

TRUE
Percentage of True values: 81.22807017543859%
